# **Black Wednesday**

### **1. Import and clean Data**

In [69]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, wilcoxon
import statsmodels.formula.api as smf
from lets_plot import *
LetsPlot.setup_html()

In [70]:
plots = []

UK_data = pd.read_excel("../data/UK.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "National_Exchange_Rate",
    "Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index": "Real_Effective_Exchange_Rate",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

Swedish_data = pd.read_excel("../data/Sweden.xlsx", skiprows=2).rename(columns={
    "Unnamed: 0": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "National_Exchange_Rate",
    "Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index": "Real_Effective_Exchange_Rate",
    "Prices, Consumer Price Index, All items, Index": "CPI"
})

def clean_and_new_vars(df):
    df_clean = df.copy()
    # Create clean copy first to avoid chained indexing
    df_clean = df_clean[~df_clean['Month'].str.match(r'^\d{4}$')]
    
    # Convert dates using .loc
    df_clean.loc[:, 'Date'] = pd.to_datetime(df_clean['Month'], format='%b %Y')
    
    # Create new variables using .loc
    df_clean.loc[:, 'Real_Exchange_Rate_Growth'] = (
        df_clean['Real_Effective_Exchange_Rate'].pct_change() * 100
    )
    
    df_clean.loc[:, "CPI_Inflation"] = (
        df_clean["CPI"].pct_change(periods=12) * 100
    )
    
    df_clean.loc[:, "Industrial_Production_Growth"] = (
        df_clean["Industrial_Production"].pct_change(periods=12) * 100
    )
    
    # Drop columns and NA values
    return (
        df_clean.drop(["Month"], axis=1)
                .dropna()
                .reset_index(drop=True)
    )

UK_data = clean_and_new_vars(UK_data)
Swedish_data = clean_and_new_vars(Swedish_data)

In [71]:
UK_data

,Industrial_Production,National_Exchange_Rate,Real_Effective_Exchange_Rate,CPI,Date,Real_Exchange_Rate_Growth,CPI_Inflation,Industrial_Production_Growth
0,63.430733,0.415835,152.957121,35.412040,1981-01-01,3.029780,13.045236,-11.855670
1,69.829448,0.435708,152.778371,35.731304,1981-02-01,-0.116863,12.459836,-8.505468
2,70.571328,0.448625,149.318729,36.267653,1981-03-01,-2.264484,12.609049,-8.313253
3,63.801673,0.459186,149.934490,37.314818,1981-04-01,0.412380,12.039896,-5.364512
4,62.596118,0.478525,150.196746,37.557448,1981-05-01,0.174914,11.740108,-6.380028
...,...,...,...,...,...,...,...,...
247,102.398828,0.695950,126.686198,83.003514,2001-08-01,-1.253315,2.046385,3.901895
248,104.925838,0.683720,127.910878,83.225448,2001-09-01,0.966704,1.626016,1.812367
249,113.935177,0.689020,127.493038,83.114481,2001-10-01,-0.326665,1.490515,6.468172
250,113.935177,0.696190,127.831377,83.114481,2001-11-01,0.265378,1.216216,3.184080


### **2. Is there evidence that the UK joining the ERM and leaving after Black Wednesday led to changes in the volatility of the monthly growth in the real exchange rate?**

In [72]:
date_joined_ERM = pd.to_datetime("Oct 1990")
date_left_ERM = pd.to_datetime("Sept 1992")

In [73]:
Real_Exchange_Rate_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Real_Exchange_Rate_Growth')) +
    ggtitle("Volatility decreased when they joined") +
    geom_line(color='#1f77b4', size=1.0) +  
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM,   color='red', size=0.5) +
    xlab('Date') +
    ylab('Real Effective Exchange Rate growth') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Plot, "Real_Exchange_Rate_Growth_Plot"))

UK_data['Volatility'] = UK_data['Real_Effective_Exchange_Rate'].rolling(window=12).std()

Real_Exchange_Rate_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='Volatility')) +
    ggtitle("and increased (briefly) when they left") +
    geom_line(color='#1f77b4', size=1.0) +
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM, color='red', size=0.5) +
    xlab('Date') +
    ylab('Volatility (12-month Rolling STD)') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Volatility_Plot, "Real_Exchange_Rate_Growth_Volatility_Plot"))
gggrid([Real_Exchange_Rate_Growth_Plot, Real_Exchange_Rate_Growth_Volatility_Plot], ncol=2, widths=[800, 800], fit=True)

In [74]:
Real_Exchange_Rate_Growth_Plot_Sweden = (
    ggplot(Swedish_data, aes(x='Date', y='Real_Exchange_Rate_Growth')) +
    ggtitle("") +
    geom_line(color='#1f77b4', size=1.0) +  
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM,   color='red', size=0.5) +
    xlab('Date') +
    ylab('Real Effective Exchange Rate growth') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Plot_Sweden, "Real_Exchange_Rate_Growth_Plot_Sweden"))

Swedish_data['Volatility'] = Swedish_data['Real_Effective_Exchange_Rate'].rolling(window=12).std()

Real_Exchange_Rate_Growth_Volatility_Plot_Sweden = (
    ggplot(Swedish_data, aes(x='Date', y='Volatility')) +
    ggtitle("") +
    geom_line(color='#1f77b4', size=1.0) +
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM, color='red', size=0.5) +
    xlab('Date') +
    ylab('Volatility (12-month Rolling STD)') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Volatility_Plot_Sweden, "Real_Exchange_Rate_Growth_Volatility_Plot_Sweden"))
gggrid([Real_Exchange_Rate_Growth_Plot_Sweden, Real_Exchange_Rate_Growth_Volatility_Plot_Sweden], ncol=2, widths=[800, 800], fit=True)

### **3. Are there any corresponding changes in the volatility of either the difference in inflation between the UK and Germany or the growth in industrial production?**

In [75]:
German_UK_inflation = pd.DataFrame()
German_UK_inflation["Date"] = UK_data["Date"]
German_UK_inflation["UK_CPI_Inflation"] = UK_data["CPI_Inflation"]

German_CPI = pd.read_excel("../data/GermanyERM.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

German_CPI = German_CPI[~German_CPI['Month'].str.match(r'^\d{4}$')]
German_CPI['Date'] = pd.to_datetime(German_CPI['Month'], format='%b %Y')
German_CPI = German_CPI.drop("Month", axis=1)

German_CPI["German_CPI_Inflation"] = German_CPI["CPI"].pct_change(periods=12) * 100  
German_CPI = German_CPI.drop("CPI", axis=1)

German_UK_inflation = pd.merge(German_UK_inflation, German_CPI, on="Date")
German_UK_inflation = German_UK_inflation.dropna()

German_UK_inflation["Difference"] = German_UK_inflation["UK_CPI_Inflation"] - German_UK_inflation["German_CPI_Inflation"]

In [76]:
UK_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='UK_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK CPI Inflation') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((UK_Inflation_Plot, "UK_Inflation_Plot"))

German_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='German_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('German CPI Inflation') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((German_Inflation_Plot, "German_Inflation_Plot"))

Difference_In_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Difference')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Difference in Inflation (UK - Germany)') +
    xlab('Date') +
    ylab('Inflation Difference (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((Difference_In_Inflation_Plot, "Difference_In_Inflation_Plot"))

gggrid([Difference_In_Inflation_Plot, UK_Inflation_Plot, German_Inflation_Plot])

In [77]:
UK_Industrial_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Industrial_Production_Growth')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK Industrial Production Growth') +
    xlab('Date') +
    ylab('Industrial Production Growth (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)

plots.append((UK_Industrial_Growth_Plot,"UK_Industrial_Growth_Plot"))

UK_Industrial_Growth_Plot

In [78]:
Swedish_Industrial_Growth_Plot = (
    ggplot(Swedish_data, aes(x='Date', y='Industrial_Production_Growth')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK Industrial Production Growth') +
    xlab('Date') +
    ylab('Industrial Production Growth (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)

plots.append((Swedish_Industrial_Growth_Plot,"Swedish_Industrial_Growth_Plot"))

Swedish_Industrial_Growth_Plot

In [79]:

German_UK_inflation["Inflation_Difference_Volatility"] = (
    German_UK_inflation["Difference"]
    .rolling(window=12)
    .std()
)

Inflation_Difference_Volatility_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Inflation_Difference_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Volatility of the Inflation Difference (UK - Germany)') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((Inflation_Difference_Volatility_Plot, "Inflation_Difference_Volatility_Plot"))

UK_data["IP_Growth_Volatility"] = (
    UK_data["Industrial_Production_Growth"]
    .rolling(window=12)
    .std()
)

IP_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='IP_Growth_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Volatility of UK IP Growth') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((IP_Growth_Volatility_Plot, "IP_Growth_Volatility_Plot"))

gggrid([IP_Growth_Volatility_Plot, Inflation_Difference_Volatility_Plot])

In [80]:
Swedish_data["IP_Growth_Volatility"] = (
    Swedish_data["Industrial_Production_Growth"]
    .rolling(window=12)
    .std()
)

Swedish_IP_Growth_Volatility_Plot = (
    ggplot(Swedish_data, aes(x='Date', y='IP_Growth_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((Swedish_IP_Growth_Volatility_Plot, "Swedish_IP_Growth_Volatility_Plot"))

### **4.Produce informative figures for the first two sections of the Black Wednesday part of the report**

In [81]:
UK_inflation_plot_before_ERM = (
    ggplot(UK_data[UK_data['Date'] <= date_joined_ERM], aes(x='Date', y='CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    ggtitle("UK Inflation was well above 2% during the years preceding the UK's entrance to the ERM") +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((UK_inflation_plot_before_ERM, "UK_inflation_plot_before_ERM"))

UK_inflation_plot_before_ERM

In [ ]:
def assign_periods(df, date_col, date_joined, date_left):
    conditions = [
        df[date_col] < date_joined,
        (df[date_col] >= date_joined) & (df[date_col] <= date_left),
        df[date_col] > date_left
    ]
    choices = ['Pre-ERM', 'During-ERM', 'Post-ERM']

    # Use a string instead of np.nan for the default
    df['Period'] = np.select(conditions, choices, default='Unknown')
    return df

def run_tests(df, measure):
    # Extract series by period, drop missing
    pre_data = df.loc[df['Period'] == 'Pre-ERM', measure].dropna()
    dur_data = df.loc[df['Period'] == 'During-ERM', measure].dropna()
    post_data = df.loc[df['Period'] == 'Post-ERM', measure].dropna()
    
    # T-tests (Welch)
    pre_dur_ttest  = ttest_ind(pre_data, dur_data, equal_var=False)
    dur_post_ttest = ttest_ind(dur_data, post_data, equal_var=False)
    
    # Wilcoxon (paired => truncate to min length)
    pre_dur_len  = min(len(pre_data), len(dur_data))
    dur_post_len = min(len(dur_data), len(post_data))
    
    pre_dur_wilcoxon  = wilcoxon(pre_data.iloc[:pre_dur_len], dur_data.iloc[:pre_dur_len])
    dur_post_wilcoxon = wilcoxon(dur_data.iloc[:dur_post_len], post_data.iloc[:dur_post_len])
    
    return {
        't-test (Pre vs During)': pre_dur_ttest,
        't-test (During vs Post)': dur_post_ttest,
        'Wilcoxon (Pre vs During)': pre_dur_wilcoxon,
        'Wilcoxon (During vs Post)': dur_post_wilcoxon
    }

# -----------------------------
# 1. Assign Periods
# -----------------------------
UK_data = assign_periods(UK_data, 'Date', date_joined_ERM, date_left_ERM)
Swedish_data = assign_periods(Swedish_data, 'Date', date_joined_ERM, date_left_ERM)

# -----------------------------
# 2. Run Volatility Tests (UK)
# -----------------------------
vol_results = run_tests(UK_data, 'Volatility')
print("UK Real Excahnge Rate Volatility Tests")
for desc, result in vol_results.items():
    print(f"{desc}: {result}")

# -----------------------------
# 3. Run IP Growth Volatility Tests (UK)
# -----------------------------
ip_results = run_tests(UK_data, 'IP_Growth_Volatility')
print("\nUK IP Growth Volatility Tests")
for desc, result in ip_results.items():
    print(f"{desc}: {result}")

# -----------------------------
# 4. Inflation Differential
# -----------------------------


# Assign periods again
df_infl = assign_periods(German_UK_inflation, 'Date', date_joined_ERM, date_left_ERM)

# Run tests on Inflation_Diff
infl_results = run_tests(df_infl, 'Inflation_Difference_Volatility')
print("\nInflation Differential (Germany - UK) Tests")
for desc, result in infl_results.items():
    print(f"{desc}: {result}")

UK Volatility Tests
t-test (Pre vs During): TtestResult(statistic=np.float64(3.79753612700855), pvalue=np.float64(0.0006208284650843367), df=np.float64(31.79655425544075))
t-test (During vs Post): TtestResult(statistic=np.float64(-1.1863748400529912), pvalue=np.float64(0.2431792844191029), df=np.float64(36.30991625771277))
Wilcoxon (Pre vs During): WilcoxonResult(statistic=np.float64(39.0), pvalue=np.float64(0.0008462667465209961))
Wilcoxon (During vs Post): WilcoxonResult(statistic=np.float64(27.0), pvalue=np.float64(0.00014960765838623047))

UK IP Growth Volatility Tests
t-test (Pre vs During): TtestResult(statistic=np.float64(0.8998944749297078), pvalue=np.float64(0.3708633807732262), df=np.float64(80.41532674277784))
t-test (During vs Post): TtestResult(statistic=np.float64(-1.954328133834658), pvalue=np.float64(0.05631745243409211), df=np.float64(49.55120357046914))
Wilcoxon (Pre vs During): WilcoxonResult(statistic=np.float64(80.0), pvalue=np.float64(0.045679688453674316))
Wilcox

In [84]:
German_UK_inflation

,Date,UK_CPI_Inflation,Industrial_Production,German_CPI_Inflation,Difference,Inflation_Difference_Volatility
0,1981-01-01,13.045236,64.239198,5.922863,7.122372,NaN
1,1981-02-01,12.459836,66.355124,5.442191,7.017645,NaN
2,1981-03-01,12.609049,71.869356,5.826557,6.782492,NaN
3,1981-04-01,12.039896,67.557355,5.795170,6.244726,NaN
4,1981-05-01,11.740108,66.282990,5.764074,5.976034,NaN
...,...,...,...,...,...,...
247,2001-08-01,2.046385,85.766839,2.097910,-0.051525,0.305048
248,2001-09-01,1.626016,90.076202,1.860469,-0.234453,0.307946
249,2001-10-01,1.490515,94.490672,1.862635,-0.372120,0.304224
250,2001-11-01,1.216216,95.016204,1.511626,-0.295410,0.277009


In [ ]:

# Calculate volatility summaries by period for each dataset
UK_volatility_summary = UK_data.groupby('Period')[['Volatility', 'IP_Growth_Volatility']].mean()
Swedish_volatility_summary = Swedish_data.groupby('Period')[['Volatility', 'IP_Growth_Volatility']].mean()

print("UK Volatility Summary:")
print(UK_volatility_summary)
print("\nSwedish Volatility Summary:")
print(Swedish_volatility_summary)

# Add a country identifier to each dataset
UK_data['Country'] = 'UK'
Swedish_data['Country'] = 'Sweden'

# Combine both datasets
combined_data = pd.concat([UK_data, Swedish_data], ignore_index=True)

# Define the Treatment variable: 1 if UK, 0 if Sweden
combined_data['Treatment'] = combined_data['Country'].apply(lambda x: 1 if x == 'UK' else 0)

# Calculate rolling volatility averages by period and country for DiD estimation
did_data = combined_data.groupby(['Country', 'Period'])[['Volatility', 'IP_Growth_Volatility']].mean().reset_index()

# Pivot data for easier comparison
volatility_did = did_data.pivot(index='Period', columns='Country', values='Volatility')
ip_growth_did = did_data.pivot(index='Period', columns='Country', values='IP_Growth_Volatility')

# ----------------------------
# Difference-in-Differences (DiD) estimates for Joining ERM (Pre-ERM vs. During-ERM)
volatility_diff_pre_during = (volatility_did.loc['During-ERM', 'UK'] - volatility_did.loc['Pre-ERM', 'UK']) - \
                             (volatility_did.loc['During-ERM', 'Sweden'] - volatility_did.loc['Pre-ERM', 'Sweden'])
ip_growth_diff_pre_during = (ip_growth_did.loc['During-ERM', 'UK'] - ip_growth_did.loc['Pre-ERM', 'UK']) - \
                            (ip_growth_did.loc['During-ERM', 'Sweden'] - ip_growth_did.loc['Pre-ERM', 'Sweden'])

print("Difference-in-Differences for Exchange Rate Volatility (Joining ERM):", volatility_diff_pre_during)
print("Difference-in-Differences for IP Growth Volatility (Joining ERM):", ip_growth_diff_pre_during)

# ----------------------------
# Difference-in-Differences (DiD) estimates for Leaving ERM (During-ERM vs. Post-ERM)
volatility_diff_during_post = (volatility_did.loc['Post-ERM', 'UK'] - volatility_did.loc['During-ERM', 'UK']) - \
                              (volatility_did.loc['Post-ERM', 'Sweden'] - volatility_did.loc['During-ERM', 'Sweden'])
ip_growth_diff_during_post = (ip_growth_did.loc['Post-ERM', 'UK'] - ip_growth_did.loc['During-ERM', 'UK']) - \
                             (ip_growth_did.loc['Post-ERM', 'Sweden'] - ip_growth_did.loc['During-ERM', 'Sweden'])

print("Difference-in-Differences for Exchange Rate Volatility (Leaving ERM):", volatility_diff_during_post)
print("Difference-in-Differences for IP Growth Volatility (Leaving ERM):", ip_growth_diff_during_post)

# ----------------------------
# DiD Regression for Joining ERM (Pre-ERM vs. During-ERM)
# Create dummy variable for During-ERM and its interaction with Treatment
combined_data['During_ERM'] = combined_data['Period'].apply(lambda x: 1 if x == 'During-ERM' else 0)
combined_data['Interaction_Joining'] = combined_data['During_ERM'] * combined_data['Treatment']

# Run DiD regression for exchange rate volatility (Joining ERM)
model_volatility_joining = smf.ols(
    formula='Volatility ~ During_ERM + Treatment + Interaction_Joining',
    data=combined_data
).fit()

# Run DiD regression for IP growth volatility (Joining ERM)
model_ip_growth_joining = smf.ols(
    formula='IP_Growth_Volatility ~ During_ERM + Treatment + Interaction_Joining',
    data=combined_data
).fit()

print(model_volatility_joining.summary())
print(model_ip_growth_joining.summary())

# ----------------------------
# DiD Regression for Leaving ERM (During-ERM vs. Post-ERM)
# Create dummy variable for Post-ERM and its interaction with Treatment
combined_data['Post_ERM'] = combined_data['Period'].apply(lambda x: 1 if x == 'Post-ERM' else 0)
combined_data['Interaction'] = combined_data['Post_ERM'] * combined_data['Treatment']

# Run DiD regression for exchange rate volatility (Leaving ERM)
model_volatility_leaving = smf.ols(
    formula='Volatility ~ Post_ERM + Treatment + Interaction',
    data=combined_data
).fit()

# Run DiD regression for IP growth volatility (Leaving ERM)
model_ip_growth_leaving = smf.ols(
    formula='IP_Growth_Volatility ~ Post_ERM + Treatment + Interaction',
    data=combined_data
).fit()

print(model_volatility_leaving.summary())
print(model_ip_growth_leaving.summary())


UK Volatility Summary:
            Volatility  IP_Growth_Volatility
Period                                      
During-ERM    2.348158              2.538486
Post-ERM      2.856892              2.742373
Pre-ERM       3.920460              2.649460

Swedish Volatility Summary:
            Volatility  IP_Growth_Volatility
Period                                      
During-ERM    1.896764              3.151348
Post-ERM      3.488537              5.837615
Pre-ERM       2.427508              2.671327
Difference-in-Differences for Exchange Rate Volatility (Joining ERM): -1.0415571595904145
Difference-in-Differences for IP Growth Volatility (Joining ERM): -0.5909939805942903
Difference-in-Differences for Exchange Rate Volatility (Leaving ERM): -1.0830398199902336
Difference-in-Differences for IP Growth Volatility (Leaving ERM): -2.4823795945540725
                            OLS Regression Results                            
Dep. Variable:             Volatility   R-squared:                 

In [ ]:
for _plot, _plot_name in plots:
    ggsave(plot=_plot, filename=f"{_plot_name}.png", path="../figures")